In [1]:
### 融资融券对股价的影响
import h5py
from jqdatasdk import *
import pandas as pd
from datetime import datetime


auth("13818134484", "12345678")
get_query_count() # 查询剩余可调用次数

提示：当前环境pandas版本为0.25，get_price与get_fundamentals_continuously接口panel参数将固定为False
注意：0.25以上版本pandas不支持panel，如使用该数据结构和相关函数请注意修改
auth success 


{'total': 10000000, 'spare': 9972664}

In [55]:
# 数据存储
def save_jqdata(stock_code, target_day_str):
    with h5py.File('D:/data_file/jqdata.hdf5', 'a') as files:

        if target_day_str in files.keys():
            grp = files[target_day_str]
        else:
            grp = files.create_group(target_day_str)

        if stock_code in grp.keys():
            subgrp = grp[stock_code]
        else:
            subgrp = grp.create_group(stock_code)

        # 每日行情数据
        if "price_daily" not in subgrp.keys():
            subgrp_daily = get_price(stock_code, start_date=target_day_str, end_date=target_day_str, frequency='daily',fq='post')
            subgrp.create_dataset("price_daily", data=subgrp_daily)
        else:
            print('写入失败.日期:{}.代码:{}.每日行情数据已存在.'.format(target_day_str, stock_code))

        # 每日分钟数据
        if "price_minute" not in subgrp.keys():
            subgrp_minute = get_price(stock_code, start_date=target_day_str, end_date=target_day_str, frequency='minute', fq='post')
            subgrp.create_dataset("price_minute", data=subgrp_minute)
        else:
            print('写入失败.日期:{}.代码:{}.每日分钟数据已存在.'.format(target_day_str, stock_code))

        # 每日集合竞价
        if "price_call" not in subgrp.keys():
            subgrp_call = get_call_auction(stock_code, start_date=target_day_str, end_date=target_day_str)
            del subgrp_call['time']
            del subgrp_call['code']
            if subgrp_call.shape[0] > 0: # 集合竞价为空
                subgrp.create_dataset("price_call", data=subgrp_call)
        else:
            print('写入失败.日期:{}.代码:{}.每日集合竞价数据已存在.'.format(target_day_str, stock_code))

        # 每日融资融券
        if "price_mtss" not in subgrp.keys():
            subgrp_mtss = get_mtss(stock_code, start_date=target_day_str, end_date=target_day_str)
            del subgrp_mtss['date']
            del subgrp_mtss['sec_code']
            if subgrp_mtss.shape[0] > 0: # 无融资融券
                subgrp.create_dataset("price_mtss", data=subgrp_mtss)
        else:
            print('写入失败.日期:{}.代码:{}.每日融资融券数据已存在.'.format(target_day_str, stock_code))
    print('执行完成.日期:{}.代码:{}.'.format(target_day_str, stock_code))


# 每日数据存储
def save_jqdata_daily(stock_list, target_day_str):
    for _code in stock_list:
        save_jqdata(stock_code=_code, target_day_str=target_day_str)

In [2]:
# 获取交易日数据
trade_days = get_all_trade_days() 
trade_days_str = [i.strftime('%Y-%m-%d') for i in list(trade_days)]

today = datetime.now()
today_str = datetime.now().strftime('%Y-%m-%d')

# 当日前一个交易日作为目标
for _, i in enumerate(trade_days_str):
    if today_str <= i:
        target_day_str = trade_days_str[_ - 1]
        break  

# 目标日与前一个交易日间隔天数
target_day_pre_delta = (trade_days[trade_days_str.index(today_str) - 1] - trade_days[trade_days_str.index(today_str) - 2]).days

In [3]:
# 获取未退市清单
stocks = get_all_securities(['stock'])
stocks['code'] = stocks.index.values

stocks['market'] = stocks[['code']].applymap(lambda x:x.split('.')[1])
stocks['stock_code'] = stocks[['code']].applymap(lambda x:x.split('.')[0])

stocks['是否退市'] = stocks[['end_date']].applymap(lambda x:1 if x < today else 0)

stock_list = stocks[stocks['是否退市'] == 0]['code'].tolist()

In [51]:
get_query_count()

{'total': 10000000, 'spare': 9989732}

In [54]:
for i in trade_days_str[trade_days_str.index(target_day_str)-28:trade_days_str.index(target_day_str)]:
    save_jqdata_daily(stock_list, i)

['2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10']

In [ ]:
with h5py.File('D:/data_file/jqdata.hdf5', 'r') as files:
    